In [41]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from collections import defaultdict  
import math  
from sys import stdout
import pickle
from evaulation import evaluate

In [42]:
def get_predict(df, pred_col, top_fill, ranknum):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= ranknum]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df 

In [43]:
def merge_label(train, label):
    tmp = pd.merge(left = train,
            right = label[['user_id','item_id','future_click']],
            how = 'left',
            on = ['user_id','item_id'])
    tmp.loc[~pd.isna(tmp['future_click']), 'future_click'] = 1
    tmp.loc[pd.isna(tmp['future_click']), 'future_click'] = 0
    return tmp

In [44]:
model1_train = pd.read_csv('../../user_data/model_1/new_recall/recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt_interactive_countdetail_userfeature_partialsim_emergency.csv')
model1_label = pd.read_csv('../../user_data/model_1/model_1_debias_track_answer.csv', 
                           names = ['phase','user_id','item_id','future_click'])
model1_train = merge_label(model1_train, model1_label)

In [45]:
model1_train.shape

(15397325, 71)

In [46]:
offline_train = pd.read_csv('../../user_data/offline/new_recall/recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt_interactive_countdetail_userfeature_partialsim_emergency.csv')
offline_label = pd.read_csv('../../user_data/offline/offline_debias_track_answer.csv', 
                            names = ['phase','user_id','item_id','future_click'])
offline_train = merge_label(offline_train, offline_label)

In [47]:
offline_train.shape

(16405037, 71)

In [48]:
offline_train = offline_train[offline_train['phrase']>6]
offline_train = offline_train.reset_index(drop=True)

In [49]:
col_sel = [x for x in offline_train.columns if x not in ['user_item_count_max_time','user_item_count_min_time',
                                                        'time_interval','item_count_4h','phrase','item_count_6h',
                                                        'is_user_count_climax','item_count_2h','is_user_count_lowerpoint',
                                                        'item_count_1h']]

In [50]:
len(col_sel)

61

# 线下

In [51]:
now_phase = 9
train_path = '../../user_data/offline/'  
test_path = '../../user_data/offline/'
header = 'offline'


item_sim_list = []
item_cnt_list = []
user_item = []

whole_click = pd.DataFrame()  
for c in range(7,now_phase + 1):  
    print('phase:', c)  
    click_train = pd.read_csv(train_path + header + '_train_click_{}_time.csv'.format(c))  
    click_test = pd.read_csv(test_path + header + '_test_click_{}_time.csv'.format(c))  
    click_query = pd.read_csv(test_path + header + '_test_qtime_{}_time.csv'.format(c)) 



    all_click = click_train.append(click_test)  
    whole_click = whole_click.append(all_click)  


    whole_click =  whole_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
    whole_click = whole_click.sort_values('time')
    whole_click = whole_click.reset_index(drop=True)

# find most popular items  
top50_click = whole_click['item_id'].value_counts().index[:500].values  
top50_click = ','.join([str(i) for i in top50_click])  

phase: 7
phase: 8
phase: 9


In [162]:
model_train = model1_train
#model_train = pd.concat([model1_train,offline_train])
#model_train = model_train.reset_index(drop=True)

model_train_p = model_train[model_train['future_click']==1]
model_train_p = model_train_p.reset_index(drop=True)

model_train_n = model_train[model_train['future_click']==0]
model_train_n = model_train_n.reset_index(drop=True)

In [164]:
model_train_p.shape

(5068, 71)

In [165]:
online_train = offline_train

In [166]:
online_train.shape

(5049093, 71)

In [167]:
import random

def generateDataset(df_n,df_p,random_seed):
    random.seed(random_seed)
    n_index = random.sample(list(range(len(df_n))), len(df_p)*5)
    df_ns = df_n.loc[n_index]
    df = pd.concat([df_ns,df_p])
    df = df.reset_index(drop=True)
    return df

model_train_s_1 = generateDataset(model_train_n,model_train_p,2020)
model_train_s_2 = generateDataset(model_train_n,model_train_p,0)
model_train_s_3 = generateDataset(model_train_n,model_train_p,2019)
model_train_s_4 = generateDataset(model_train_n,model_train_p,1000)
model_train_s_5 = generateDataset(model_train_n,model_train_p,3000)
model_train_s_6 = generateDataset(model_train_n,model_train_p,2021)

In [168]:
def addWeightForDataSet(df,item_degree_median,weight):
    df['sample_weight'] = df['count']/item_degree_median
    df['sample_weight'] = df['sample_weight'].apply(lambda x: 5 if x<1 else 1)
    df.loc[(df['count']<item_degree_median)&(df['future_click']==1),'sample_weight'] = weight
    df.loc[(df['count']<item_degree_median)&(df['future_click']==1)&(df['phrase'].isin([7,8,9])), 'sample_weight'] = weight * 2
    return df

In [169]:
model_train_s_1 = addWeightForDataSet(model_train_s_1,30,35)
model_train_s_2 = addWeightForDataSet(model_train_s_2,30,35)
model_train_s_3 = addWeightForDataSet(model_train_s_3,30,35)
model_train_s_4 = addWeightForDataSet(model_train_s_4,30,35)
model_train_s_5 = addWeightForDataSet(model_train_s_5,30,35)
model_train_s_6 = addWeightForDataSet(model_train_s_6,30,35)

In [170]:
feature_list = [x for x in col_sel if x not in ['user_id','item_id','future_click','sample_weight'] 
                and 'result' not in x]

In [171]:
len(feature_list)

58

In [172]:
feature_list

['sim',
 'feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'cn_sim',
 'hpi_sim',
 'hdi_sim',
 'lhn1_sim',
 'ra_sim',
 'aa_sim',
 'item_count_12h',
 'count',
 'diff_from_last',
 'diff_from_next',
 'diff_from_max',
 'nn',
 'txt_feature_0',
 'txt_feature_1',
 'txt_feature_2',
 'txt_feature_3',
 'txt_feature_4',
 'img_feature_0',
 'img_feature_1',
 'img_feature_2',
 'img_feature_3',
 'img_feature_4',
 'item_count',
 'user_item_count_cnt',
 'user_item_count_ratio',
 'user_item_count_timedelta',
 'item_max_time_in_phrase',
 'is_climix',
 'is_lowest_point',
 'item_diff_from_min',
 'item_diff_from_mean',
 'item_count_rankin_phrase',
 'item_count_rankin_history',
 'item_never_in_phrase',
 'user_mean_count',
 'user_max_count',
 'user_min_count',
 'sim_partial',
 'feature_0_partial',
 'feature_1_partial',
 'feature_2_partial',
 'feature_3_partial',
 'emergency_feature_0',
 'emergency_feature_1

In [173]:
feature_list_noleak = [x for x in col_sel if x not in ['user_id','item_id','future_click','sample_weight','result',
                                                       'diff_from_next'] and 'result' not in x]

In [174]:
len(feature_list_noleak)

57

In [175]:
def cbt_model(m,df_train,df_test,feat):
    m.fit(df_train[feat],df_train[['future_click']],sample_weight=list(df_train['sample_weight']))
    print(sorted(dict(zip(m.feature_names_,m.feature_importances_)).items(), key=lambda x:x[1], reverse=True))
    result = m.predict_proba(df_test[feat])[:,1]
    return result

In [176]:
df_res = pd.DataFrame()

In [177]:
import catboost as cat
clf_cbt = cat.CatBoostClassifier(iterations=2500,learning_rate=0.01,depth=6,
                                   verbose=True,thread_count=12,colsample_bylevel=0.8
                                   ,l2_leaf_reg=1
                                   ,random_seed=1024)

df_res['result_1'] = cbt_model(clf_cbt,model_train_s_1,online_train,feature_list)

df_res['result_2'] = cbt_model(clf_cbt,model_train_s_2,online_train,feature_list)

df_res['result_3'] = cbt_model(clf_cbt,model_train_s_3,online_train,feature_list)

df_res['result_4'] = cbt_model(clf_cbt,model_train_s_4,online_train,feature_list)

df_res['result_5'] = cbt_model(clf_cbt,model_train_s_5,online_train,feature_list)

df_res['result_6'] = cbt_model(clf_cbt,model_train_s_6,online_train,feature_list)


0:	learn: 0.6811937	total: 12.4ms	remaining: 24.9s
1:	learn: 0.6704701	total: 24.6ms	remaining: 24.6s
2:	learn: 0.6563659	total: 37.8ms	remaining: 25.1s
3:	learn: 0.6451964	total: 51ms	remaining: 25.4s
4:	learn: 0.6322468	total: 63.7ms	remaining: 25.4s
5:	learn: 0.6203855	total: 75.7ms	remaining: 25.2s
6:	learn: 0.6091465	total: 88.9ms	remaining: 25.3s
7:	learn: 0.5989847	total: 102ms	remaining: 25.3s
8:	learn: 0.5916641	total: 115ms	remaining: 25.5s
9:	learn: 0.5818028	total: 128ms	remaining: 25.4s
10:	learn: 0.5718055	total: 140ms	remaining: 25.4s
11:	learn: 0.5634617	total: 152ms	remaining: 25.2s
12:	learn: 0.5549263	total: 169ms	remaining: 25.8s
13:	learn: 0.5465548	total: 183ms	remaining: 26s
14:	learn: 0.5376156	total: 196ms	remaining: 25.9s
15:	learn: 0.5305146	total: 209ms	remaining: 25.9s
16:	learn: 0.5232159	total: 221ms	remaining: 25.8s
17:	learn: 0.5176546	total: 234ms	remaining: 25.7s
18:	learn: 0.5107184	total: 246ms	remaining: 25.6s
19:	learn: 0.5039155	total: 258ms	rema

174:	learn: 0.2650281	total: 2.26s	remaining: 23.6s
175:	learn: 0.2647183	total: 2.27s	remaining: 23.6s
176:	learn: 0.2644719	total: 2.29s	remaining: 23.6s
177:	learn: 0.2641339	total: 2.3s	remaining: 23.6s
178:	learn: 0.2638410	total: 2.31s	remaining: 23.6s
179:	learn: 0.2635864	total: 2.33s	remaining: 23.5s
180:	learn: 0.2632240	total: 2.34s	remaining: 23.5s
181:	learn: 0.2628959	total: 2.35s	remaining: 23.5s
182:	learn: 0.2626334	total: 2.37s	remaining: 23.5s
183:	learn: 0.2622443	total: 2.38s	remaining: 23.5s
184:	learn: 0.2617712	total: 2.4s	remaining: 23.5s
185:	learn: 0.2614075	total: 2.41s	remaining: 23.5s
186:	learn: 0.2612557	total: 2.42s	remaining: 23.5s
187:	learn: 0.2610672	total: 2.44s	remaining: 23.5s
188:	learn: 0.2608197	total: 2.45s	remaining: 23.5s
189:	learn: 0.2605094	total: 2.46s	remaining: 23.5s
190:	learn: 0.2602333	total: 2.48s	remaining: 23.5s
191:	learn: 0.2599749	total: 2.49s	remaining: 23.5s
192:	learn: 0.2597857	total: 2.5s	remaining: 23.5s
193:	learn: 0.2

339:	learn: 0.2370477	total: 4.53s	remaining: 22.1s
340:	learn: 0.2369983	total: 4.54s	remaining: 22.1s
341:	learn: 0.2368571	total: 4.55s	remaining: 22.1s
342:	learn: 0.2367463	total: 4.57s	remaining: 22.1s
343:	learn: 0.2366318	total: 4.58s	remaining: 22.1s
344:	learn: 0.2365672	total: 4.6s	remaining: 22.1s
345:	learn: 0.2365090	total: 4.61s	remaining: 22s
346:	learn: 0.2363641	total: 4.63s	remaining: 22s
347:	learn: 0.2362501	total: 4.64s	remaining: 22s
348:	learn: 0.2360970	total: 4.65s	remaining: 22s
349:	learn: 0.2359765	total: 4.67s	remaining: 22s
350:	learn: 0.2359266	total: 4.68s	remaining: 22s
351:	learn: 0.2357700	total: 4.7s	remaining: 22s
352:	learn: 0.2357316	total: 4.71s	remaining: 22s
353:	learn: 0.2356676	total: 4.72s	remaining: 22s
354:	learn: 0.2355952	total: 4.74s	remaining: 22s
355:	learn: 0.2355164	total: 4.75s	remaining: 21.9s
356:	learn: 0.2354688	total: 4.76s	remaining: 21.9s
357:	learn: 0.2353735	total: 4.78s	remaining: 21.9s
358:	learn: 0.2352605	total: 4.79s

504:	learn: 0.2244148	total: 6.84s	remaining: 20.2s
505:	learn: 0.2242812	total: 6.85s	remaining: 20.2s
506:	learn: 0.2242343	total: 6.87s	remaining: 20.2s
507:	learn: 0.2241990	total: 6.88s	remaining: 20.2s
508:	learn: 0.2241282	total: 6.89s	remaining: 20.2s
509:	learn: 0.2240661	total: 6.91s	remaining: 20.2s
510:	learn: 0.2239957	total: 6.92s	remaining: 20.2s
511:	learn: 0.2239621	total: 6.93s	remaining: 20.2s
512:	learn: 0.2239242	total: 6.95s	remaining: 20.1s
513:	learn: 0.2238945	total: 6.96s	remaining: 20.1s
514:	learn: 0.2238456	total: 6.98s	remaining: 20.1s
515:	learn: 0.2237802	total: 6.99s	remaining: 20.1s
516:	learn: 0.2236992	total: 7s	remaining: 20.1s
517:	learn: 0.2236476	total: 7.02s	remaining: 20.1s
518:	learn: 0.2235591	total: 7.03s	remaining: 20.1s
519:	learn: 0.2235117	total: 7.05s	remaining: 20.1s
520:	learn: 0.2234770	total: 7.06s	remaining: 20s
521:	learn: 0.2233919	total: 7.08s	remaining: 20s
522:	learn: 0.2233554	total: 7.09s	remaining: 20s
523:	learn: 0.2232802

669:	learn: 0.2154938	total: 9.14s	remaining: 18.1s
670:	learn: 0.2154693	total: 9.16s	remaining: 18.1s
671:	learn: 0.2154003	total: 9.17s	remaining: 18.1s
672:	learn: 0.2153171	total: 9.18s	remaining: 18.1s
673:	learn: 0.2152749	total: 9.2s	remaining: 18.1s
674:	learn: 0.2152339	total: 9.21s	remaining: 18.1s
675:	learn: 0.2152100	total: 9.23s	remaining: 18.1s
676:	learn: 0.2151339	total: 9.24s	remaining: 18.1s
677:	learn: 0.2150901	total: 9.25s	remaining: 18s
678:	learn: 0.2150636	total: 9.27s	remaining: 18s
679:	learn: 0.2150335	total: 9.28s	remaining: 18s
680:	learn: 0.2150130	total: 9.29s	remaining: 18s
681:	learn: 0.2149719	total: 9.31s	remaining: 18s
682:	learn: 0.2149252	total: 9.32s	remaining: 18s
683:	learn: 0.2148657	total: 9.34s	remaining: 18s
684:	learn: 0.2147991	total: 9.35s	remaining: 18s
685:	learn: 0.2147516	total: 9.37s	remaining: 17.9s
686:	learn: 0.2147216	total: 9.38s	remaining: 17.9s
687:	learn: 0.2146724	total: 9.39s	remaining: 17.9s
688:	learn: 0.2146253	total: 

...

1351:	learn: 0.1745881	total: 16.2s	remaining: 7.76s
1352:	learn: 0.1745409	total: 16.2s	remaining: 7.75s
1353:	learn: 0.1744927	total: 16.2s	remaining: 7.74s
1354:	learn: 0.1744439	total: 16.2s	remaining: 7.73s
1355:	learn: 0.1743867	total: 16.2s	remaining: 7.72s
1356:	learn: 0.1743317	total: 16.3s	remaining: 7.7s
1357:	learn: 0.1742630	total: 16.3s	remaining: 7.69s
1358:	learn: 0.1742173	total: 16.3s	remaining: 7.68s
1359:	learn: 0.1741532	total: 16.3s	remaining: 7.67s
1360:	learn: 0.1741062	total: 16.3s	remaining: 7.66s
1361:	learn: 0.1740613	total: 16.3s	remaining: 7.65s
1362:	learn: 0.1740115	total: 16.3s	remaining: 7.63s
1363:	learn: 0.1739548	total: 16.3s	remaining: 7.62s
1364:	learn: 0.1738981	total: 16.4s	remaining: 7.61s
1365:	learn: 0.1738391	total: 16.4s	remaining: 7.6s
1366:	learn: 0.1737872	total: 16.4s	remaining: 7.59s
1367:	learn: 0.1737180	total: 16.4s	remaining: 7.58s
1368:	learn: 0.1736897	total: 16.4s	remaining: 7.56s
1369:	learn: 0.1736139	total: 16.4s	remaining: 7

1511:	learn: 0.1649109	total: 18.2s	remaining: 5.88s
1512:	learn: 0.1648659	total: 18.2s	remaining: 5.86s
1513:	learn: 0.1648018	total: 18.2s	remaining: 5.85s
1514:	learn: 0.1647498	total: 18.2s	remaining: 5.84s
1515:	learn: 0.1646773	total: 18.3s	remaining: 5.83s
1516:	learn: 0.1646062	total: 18.3s	remaining: 5.82s
1517:	learn: 0.1645169	total: 18.3s	remaining: 5.8s
1518:	learn: 0.1644520	total: 18.3s	remaining: 5.79s
1519:	learn: 0.1644112	total: 18.3s	remaining: 5.78s
1520:	learn: 0.1643704	total: 18.3s	remaining: 5.77s
1521:	learn: 0.1643317	total: 18.3s	remaining: 5.76s
1522:	learn: 0.1642856	total: 18.3s	remaining: 5.74s
1523:	learn: 0.1642227	total: 18.4s	remaining: 5.73s
1524:	learn: 0.1641685	total: 18.4s	remaining: 5.72s
1525:	learn: 0.1641039	total: 18.4s	remaining: 5.71s
1526:	learn: 0.1640391	total: 18.4s	remaining: 5.7s
1527:	learn: 0.1639826	total: 18.4s	remaining: 5.68s
1528:	learn: 0.1639106	total: 18.4s	remaining: 5.67s
1529:	learn: 0.1638582	total: 18.4s	remaining: 5

1671:	learn: 0.1561142	total: 20.2s	remaining: 3.96s
1672:	learn: 0.1560439	total: 20.2s	remaining: 3.95s
1673:	learn: 0.1559952	total: 20.2s	remaining: 3.94s
1674:	learn: 0.1559602	total: 20.3s	remaining: 3.93s
1675:	learn: 0.1559221	total: 20.3s	remaining: 3.92s
1676:	learn: 0.1558834	total: 20.3s	remaining: 3.9s
1677:	learn: 0.1558191	total: 20.3s	remaining: 3.89s
1678:	learn: 0.1557520	total: 20.3s	remaining: 3.88s
1679:	learn: 0.1557022	total: 20.3s	remaining: 3.87s
1680:	learn: 0.1556568	total: 20.3s	remaining: 3.86s
1681:	learn: 0.1556138	total: 20.3s	remaining: 3.84s
1682:	learn: 0.1555689	total: 20.3s	remaining: 3.83s
1683:	learn: 0.1555024	total: 20.4s	remaining: 3.82s
1684:	learn: 0.1554533	total: 20.4s	remaining: 3.81s
1685:	learn: 0.1553898	total: 20.4s	remaining: 3.8s
1686:	learn: 0.1553500	total: 20.4s	remaining: 3.78s
1687:	learn: 0.1553106	total: 20.4s	remaining: 3.77s
1688:	learn: 0.1552817	total: 20.4s	remaining: 3.76s
1689:	learn: 0.1552097	total: 20.4s	remaining: 3

1827:	learn: 0.1485458	total: 22s	remaining: 2.07s
1828:	learn: 0.1484957	total: 22s	remaining: 2.06s
1829:	learn: 0.1484379	total: 22.1s	remaining: 2.05s
1830:	learn: 0.1483951	total: 22.1s	remaining: 2.04s
1831:	learn: 0.1483564	total: 22.1s	remaining: 2.02s
1832:	learn: 0.1483090	total: 22.1s	remaining: 2.01s
1833:	learn: 0.1482633	total: 22.1s	remaining: 2s
1834:	learn: 0.1482114	total: 22.1s	remaining: 1.99s
1835:	learn: 0.1481533	total: 22.1s	remaining: 1.98s
1836:	learn: 0.1481133	total: 22.1s	remaining: 1.96s
1837:	learn: 0.1480917	total: 22.1s	remaining: 1.95s
1838:	learn: 0.1480566	total: 22.1s	remaining: 1.94s
1839:	learn: 0.1480097	total: 22.2s	remaining: 1.93s
1840:	learn: 0.1479637	total: 22.2s	remaining: 1.91s
1841:	learn: 0.1478999	total: 22.2s	remaining: 1.9s
1842:	learn: 0.1478492	total: 22.2s	remaining: 1.89s
1843:	learn: 0.1477889	total: 22.2s	remaining: 1.88s
1844:	learn: 0.1477598	total: 22.2s	remaining: 1.87s
1845:	learn: 0.1477193	total: 22.2s	remaining: 1.85s
1

1998:	learn: 0.1409898	total: 23.9s	remaining: 11.9ms
1999:	learn: 0.1409478	total: 23.9s	remaining: 0us
[('is_climix', 10.056821102747291), ('is_lowest_point', 5.9734624847812166), ('item_count_rankin_history', 5.2968425244563475), ('nn', 4.818092902585948), ('item_never_in_phrase', 4.513586621815747), ('item_count', 4.41540299550309), ('item_diff_from_mean', 4.268352635479945), ('feature_9', 4.124299281296041), ('item_count_rankin_phrase', 3.770351751076798), ('feature_7', 3.571438362834511), ('txt_feature_0', 3.2892730437614333), ('user_mean_count', 2.63021644620639), ('user_min_count', 2.570546430650283), ('emergency_feature_3', 2.052633352833936), ('emergency_feature_2', 2.0069113793529976), ('item_max_time_in_phrase', 2.0059343910347094), ('feature_2', 1.9585851976404305), ('count', 1.9163094727141445), ('user_max_count', 1.5701352175152055), ('diff_from_next', 1.5566863013044991), ('lhn1_sim', 1.4948524941437995), ('item_diff_from_min', 1.3299935993328629), ('emergency_feature_1

In [178]:
df_res['phrase'] = online_train['phrase']
df_res['user_id'] = online_train['user_id']
df_res['item_id'] = online_train['item_id']

In [179]:
df_res['result'] = df_res['result_1'] + \
df_res['result_2'] + \
df_res['result_3'] + \
df_res['result_4'] + \
df_res['result_5'] + \
df_res['result_6'] 

In [180]:
recom_df = df_res[['user_id','item_id','result']]
result = get_predict(recom_df, 'result', top50_click, 50) 
result['user_id'] = result['user_id'].astype(int)
result.to_csv('rank_offline.csv', index=False, header=None)

print(evaluate(stdout, 'rank_offline.csv',
             answer_fname='../../user_data/offline/offline_debias_track_answer.csv', rank_num=50))


current_time: 1591868159
date_time: 2020-06-11 17:35:59
current_phase: 9
[0.09353349102898692, 0.11100779069411837, 0.20928865373309818, 0.18202764976958524]
[0.09356756267356249, 0.11773992597015259, 0.2127039627039627, 0.2094361334867664]
[0.07453767703075809, 0.0857016423183076, 0.18291944276196245, 0.14819277108433734]
[0.2616387605667114, 0.2616387605667114, 0.31444936990737915, 0.6049120426177979, 0.5396565794944763]


In [181]:
def lgb_model(df_train,df_test,feat,params,num_round):
    train_data = lgb.Dataset(df_train[feat], 
                         label=df_train[['future_click']],weight=df_train['sample_weight'])  
    print('lgb training')
    bst = lgb.train(params,
                train_data,
                num_round)    
    print('lgb predicting')
    result = bst.predict(df_test[feat])    
    return result

In [191]:
import lightgbm as lgb
import time

num_round = 2500
params = {
        'learning_rate': 0.01,
        'boosting_type': 'dart',
        'objective': 'binary',
        #'metric': 'auc',
        'max_depth': 6,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'seed': 1,
        'bagging_seed': 10,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 12,
        'verbose': 1,
    }

In [192]:
df_res['lgb_dart_1'] = lgb_model(model_train_s_1,online_train,feature_list,params,num_round)

df_res['lgb_dart_2'] = lgb_model(model_train_s_2,online_train,feature_list,params,num_round)

df_res['lgb_dart_3'] = lgb_model(model_train_s_3,online_train,feature_list,params,num_round)

df_res['lgb_dart_4'] = lgb_model(model_train_s_4,online_train,feature_list,params,num_round)

df_res['lgb_dart_5'] = lgb_model(model_train_s_5,online_train,feature_list,params,num_round)

df_res['lgb_dart_6'] = lgb_model(model_train_s_6,online_train,feature_list,params,num_round)


lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting
lgb training
lgb predicting


In [193]:
df_res['result_lgb_dart'] = df_res['lgb_dart_1'] + \
df_res['lgb_dart_2'] + \
df_res['lgb_dart_3'] + \
df_res['lgb_dart_4'] + \
df_res['lgb_dart_5'] + \
df_res['lgb_dart_6'] 

In [194]:
recom_df = df_res[['user_id','item_id','result_lgb_dart']]
result = get_predict(recom_df, 'result_lgb_dart', top50_click, 50) 
result['user_id'] = result['user_id'].astype(int)
result.to_csv('rank_offline.csv', index=False, header=None)

print(evaluate(stdout, 'rank_offline.csv',
             answer_fname='../../user_data/offline/offline_debias_track_answer.csv', rank_num=50))


current_time: 1591869516
date_time: 2020-06-11 17:58:36
current_phase: 9
[0.09240675371837556, 0.10680565804800286, 0.2069370958259847, 0.18202764976958524]
[0.09601175612482923, 0.11926667020455717, 0.2162004662004662, 0.20828538550057538]
[0.0756483135162347, 0.08404736572449709, 0.17746820109024833, 0.14216867469879518]
[0.26406681537628174, 0.26406681537628174, 0.3101196885108948, 0.6006057262420654, 0.5324816703796387]


In [195]:
df_res['result'] = df_res['result']/6
df_res['result_lgb_dart'] = df_res['result_lgb_dart']/6

In [196]:
df_res['count_na'] = online_train['count'].apply(lambda x: np.nan if x ==0 else x)
df_res['m'] = df_res['count_na'].apply(lambda x:max(0.61,1/math.log1p(x+1)))

In [197]:
df_res['result_PostProcess'] = df_res['result'] * df_res['m']
df_res['result_lgb_dart_PostProcess'] = df_res['result_lgb_dart'] * df_res['m']

In [198]:
df_res['ensemble1'] = 10 / ( 6/df_res['result_PostProcess'] + 4/df_res['result_lgb_dart_PostProcess']) 
df_res['ensemble2'] = np.power( df_res['result_PostProcess']**6 * df_res['result_lgb_dart_PostProcess']**4 , 1/10) 

In [199]:
df_res['ensemble'] = df_res['ensemble1']  + df_res['ensemble2'] 

In [200]:
#df_res['PostProcess'] = df_res['ensemble'] * df_res['m']

recom_df = df_res[['user_id','item_id','ensemble']]
result = get_predict(recom_df, 'ensemble', top50_click, 50) 
result['user_id'] = result['user_id'].astype(int)
result.to_csv('rank_offline.csv', index=False, header=None)

print(evaluate(stdout, 'rank_offline.csv',
             answer_fname='../../user_data/offline/offline_debias_track_answer.csv', rank_num=50))


current_time: 1591869530
date_time: 2020-06-11 17:58:50
current_phase: 9
[0.09390052886462799, 0.11152539865847005, 0.21105232216343328, 0.18317972350230416]
[0.09401496563777403, 0.11870115569284223, 0.2127039627039627, 0.20828538550057538]
[0.07631566961846435, 0.08764072653066847, 0.18291944276196245, 0.14819277108433734]
[0.2642311751842499, 0.2642311751842499, 0.3178672790527344, 0.6066757440567017, 0.5396578907966614]
